In [ ]:
import numpyro
from Model.Energy import SineMultivariateVonMisesEnergy
from Model.Proposal import GaussianProposal, MultivariateVonMisesProposal, UniformProposal
from Data import get_dataloader_from_data
import matplotlib.pyplot as plt
import numpy as np
import torch 
from torch.utils.data import TensorDataset, DataLoader
from Model.Trainer import SNLTrainer
import os

In [ ]:
# Set all the seed :

np.random.seed(0)
torch.manual_seed(0)

# Target Distribution 

In [ ]:
phi = torch.tensor([0.0, np.pi/2, np.pi])
kappa = torch.tensor([4.0, 10.0, 1.0])
lam = torch.zeros((3,3))
lam[0,1] = 4.0
lam[0,2] = -4.0
lam[1,2] = 10.0
lam = (lam + lam.t())


target_distribution = MultivariateVonMisesProposal(input_size=torch.Size([3]), phi=phi, kappa=kappa, lam=lam)

In [ ]:
target_sample = target_distribution.sample(10000)

In [ ]:

target_sample_train = target_sample[:6000]
target_sample_val = target_sample[6000:8000]
target_sample_test = target_sample[8000:]

data_dir = 'Data/MultivariateVonMisesData/'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

np.save(data_dir + 'target_sample_train.npy', target_sample_train.numpy())
np.save(data_dir + 'target_sample_val.npy', target_sample_val.numpy())
np.save(data_dir + 'target_sample_test.npy', target_sample_test.numpy())

fig, axs = plt.subplots(3, 3, figsize=(15, 15))
for i in range(3):
    for j in range(3):
        if i == j :
            axs[i,i].hist(target_sample_train[:, i], bins=100)
            # axs[i,i].set_title(f" {i} dimension")
        else :
            axs[i,j].scatter(target_sample_train[:, i], target_sample_train[:, j], s=1)
            axs[i,j].set_xlim(-np.pi, np.pi)
            axs[i,j].set_ylim(-np.pi, np.pi)
            # axs[i,j].set_title(f"{i} and {j} dimensions")

data_train = get_dataloader_from_data(torch.from_numpy(target_sample_train.numpy()), batch_size=256)


# Proposal Distribution

In [ ]:
proposal_distribution = GaussianProposal(input_size=torch.Size([3]), mu=torch.tensor([0.0, 0, 0]), sigma=torch.tensor([2.0, 2.0, 2.0]))
proposal_distribution = UniformProposal(input_size=torch.Size([3]), min=torch.tensor([-np.pi,-np.pi,-np.pi] ), max=torch.tensor([np.pi,np.pi,np.pi]))

In [ ]:
sample = proposal_distribution.sample(10000)
print(sample)

for i in range(3):
    for j in range(i+1, 3):
        plt.figure()
        plt.scatter(sample[:, i], sample[:, j], s=1)
        plt.axis('equal')
        plt.xlim(-np.pi, np.pi)
        plt.ylim(-np.pi, np.pi)
        plt.title(f"Scatter plot of samples from the target distribution in the {i} and {j} dimensions")
        plt.show()
        

# Energy 

In [ ]:
energy_model = SineMultivariateVonMisesEnergy(dim=3, learn_theta=True, learn_kappa=True, learn_lambda=True)

In [ ]:

mean, new_kappa, new_lam = energy_model.score_matching_estimator(target_sample_train[:100])

print(phi)
print(mean)

print('=====')
print(kappa)
print(new_kappa)

print("=====")
print(lam)
print(new_lam)




In [ ]:
mean, new_kappa, new_lam = energy_model.score_matching_estimator(target_sample_train[:1000])

print(phi)
print(mean)

print('=====')
print(kappa)
print(new_kappa)

print("=====")
print(lam)
print(new_lam)



In [ ]:
mean, new_kappa, new_lam = energy_model.score_matching_estimator(target_sample_train)

print(phi)
print(mean)

print('=====')
print(kappa)
print(new_kappa)

print("=====")
print(lam)
print(new_lam)



# Trainer 

In [ ]:
trainer = SNLTrainer(energy_model, proposal_distribution, data_train, data_train, lr=1e-2)

In [ ]:
# energy_model.theta.data = phi
# energy_model.log_kappa.data = kappa
# energy_model.lam.data = lam

In [ ]:
trainer.train(n_iter = 200000, n_iter_pretrain= 1000)